In [3]:
import findspark
findspark.init()

import pyspark
import random

print('init')

sc = pyspark.SparkContext(appName="Pi")
num_samples = 10000

def inside(p):     
  x, y = random.random(), random.random()
  return x*x + y*y < 1

count = sc.parallelize(range(0, num_samples)).filter(inside).count()
print(sc)

pi = 4 * count / num_samples
print(pi)

sc.stop()


init
<SparkContext master=local[*] appName=Pi>
3.1436


In [4]:
from pyspark.sql import SparkSession

spark = SparkSession \
    .builder \
    .appName("Python Spark SQL basic example") \
    .config("spark.some.config.option", "some-value") \
    .getOrCreate()

spark

In [5]:
# spark is an existing SparkSession
df = spark.read.csv("../nlp/stanfordSentimentTreebank/dictionary_sm.txt")
# Displays the content of the DataFrame to stdout
df.show()

df2 = spark.read.csv("../nlp/stanfordSentimentTreebank/sentiment_labels_sm.txt")
df2.show()


+--------------------+
|                 _c0|
+--------------------+
|             text|id|
|                 !|0|
|           ! '|22935|
|          ! ''|18235|
|       ! Alas|179257|
|   ! Brilliant|22936|
| ! Brilliant !|40532|
|! Brilliant ! '|2...|
|       ! C'mon|60624|
|! Gollum 's ` per...|
|               ! Oh |
|  ! Romething|140882|
|        ! Run|179259|
|   ! The Movie|60625|
|! The camera twir...|
|! True Hollywood ...|
|        ! Wow|179261|
|     ! Zoom !|179262|
|           !?|220445|
|         !? '|220446|
+--------------------+
only showing top 20 rows

+------------+
|         _c0|
+------------+
|id|sentiment|
|       0|0.5|
|       1|0.5|
|   2|0.44444|
|       3|0.5|
|   4|0.42708|
|     5|0.375|
|   6|0.41667|
|   7|0.54167|
|   8|0.33333|
|   9|0.45833|
|  10|0.47222|
|  11|0.59722|
|  12|0.33333|
|  13|0.93056|
|  14|0.80556|
|  15|0.81944|
|  16|0.76389|
|      17|0.5|
|      18|0.5|
+------------+
only showing top 20 rows



In [45]:
df = spark.read.load("../nlp/stanfordSentimentTreebank/dictionary_sm.txt",
                     format="csv", sep="|", inferSchema="true", header="true")
df.show()


+--------------------+------+
|                text|    id|
+--------------------+------+
|                   !|     0|
|                 ! '| 22935|
|                ! ''| 18235|
|              ! Alas|179257|
|         ! Brilliant| 22936|
|       ! Brilliant !| 40532|
|     ! Brilliant ! '| 22937|
|             ! C'mon| 60624|
|! Gollum 's ` per...| 13402|
|! Oh , look at th...|179258|
|         ! Romething|140882|
|               ! Run|179259|
|         ! The Movie| 60625|
|! The camera twir...|179260|
|! True Hollywood ...|140883|
|               ! Wow|179261|
|            ! Zoom !|179262|
|                  !?|220445|
|                !? '|220446|
|                   #| 60626|
+--------------------+------+
only showing top 20 rows



In [46]:
df.printSchema()
df.createOrReplaceGlobalTempView("sentences")
spark.sql("SELECT * FROM global_temp.sentences").show()


root
 |-- text: string (nullable = true)
 |-- id: integer (nullable = true)

+--------------------+------+
|                text|    id|
+--------------------+------+
|                   !|     0|
|                 ! '| 22935|
|                ! ''| 18235|
|              ! Alas|179257|
|         ! Brilliant| 22936|
|       ! Brilliant !| 40532|
|     ! Brilliant ! '| 22937|
|             ! C'mon| 60624|
|! Gollum 's ` per...| 13402|
|! Oh , look at th...|179258|
|         ! Romething|140882|
|               ! Run|179259|
|         ! The Movie| 60625|
|! The camera twir...|179260|
|! True Hollywood ...|140883|
|               ! Wow|179261|
|            ! Zoom !|179262|
|                  !?|220445|
|                !? '|220446|
|                   #| 60626|
+--------------------+------+
only showing top 20 rows



In [10]:
from pyspark.sql.types import *

sc = spark.sparkContext

# Load a text file and convert each line to a Row.
lines = sc.textFile("../nlp/stanfordSentimentTreebank/dictionary_sm.txt")
parts = lines.map(lambda l: l.split("|"))
# Each line is converted to a tuple.
wordid = parts.map(lambda p: (p[0].strip(), p[1].strip()))

# The schema is encoded in a string.
schemaString = "word id"

fields = [StructField(field_name, StringType(), True) for field_name in schemaString.split()]
schema = StructType(fields)

# Apply the schema to the RDD.
schemaWord = spark.createDataFrame(wordid, schema)

# Creates a temporary view using the DataFrame
schemaWord.createOrReplaceTempView("word")

# SQL can be run over DataFrames that have been registered as a table.
results = spark.sql("SELECT * FROM word")

results.show()


+--------------------+------+
|                word|    id|
+--------------------+------+
|                text|    id|
|                   !|     0|
|                 ! '| 22935|
|                ! ''| 18235|
|              ! Alas|179257|
|         ! Brilliant| 22936|
|       ! Brilliant !| 40532|
|     ! Brilliant ! '| 22937|
|             ! C'mon| 60624|
|! Gollum 's ` per...| 13402|
|! Oh , look at th...|179258|
|         ! Romething|140882|
|               ! Run|179259|
|         ! The Movie| 60625|
|! The camera twir...|179260|
|! True Hollywood ...|140883|
|               ! Wow|179261|
|            ! Zoom !|179262|
|                  !?|220445|
|                !? '|220446|
+--------------------+------+
only showing top 20 rows



In [47]:
df.write.mode("overwrite").saveAsTable("saved_words2")

spark.sql("show databases").show()
spark.sql("show tables").show()
spark.sql("select * from saved_words2").show()


+------------+
|databaseName|
+------------+
|     default|
+------------+

+--------+------------+-----------+
|database|   tableName|isTemporary|
+--------+------------+-----------+
| default|saved_words2|      false|
|        | parquetfile|       true|
|        |        word|       true|
+--------+------------+-----------+

+--------------------+------+
|                text|    id|
+--------------------+------+
|                   !|     0|
|                 ! '| 22935|
|                ! ''| 18235|
|              ! Alas|179257|
|         ! Brilliant| 22936|
|       ! Brilliant !| 40532|
|     ! Brilliant ! '| 22937|
|             ! C'mon| 60624|
|! Gollum 's ` per...| 13402|
|! Oh , look at th...|179258|
|         ! Romething|140882|
|               ! Run|179259|
|         ! The Movie| 60625|
|! The camera twir...|179260|
|! True Hollywood ...|140883|
|               ! Wow|179261|
|            ! Zoom !|179262|
|                  !?|220445|
|                !? '|220446|
|          

In [48]:
df.write.mode("overwrite").parquet("words.parquet")
parquetFile = spark.read.parquet("words.parquet")
parquetFile.createOrReplaceTempView("parquetFile")
words = spark.sql("SELECT * FROM parquetFile ")
words.show()
words2 = spark.sql("SELECT * FROM parquetFile ")
words2.show()


+--------------------+------+
|                text|    id|
+--------------------+------+
|                   !|     0|
|                 ! '| 22935|
|                ! ''| 18235|
|              ! Alas|179257|
|         ! Brilliant| 22936|
|       ! Brilliant !| 40532|
|     ! Brilliant ! '| 22937|
|             ! C'mon| 60624|
|! Gollum 's ` per...| 13402|
|! Oh , look at th...|179258|
|         ! Romething|140882|
|               ! Run|179259|
|         ! The Movie| 60625|
|! The camera twir...|179260|
|! True Hollywood ...|140883|
|               ! Wow|179261|
|            ! Zoom !|179262|
|                  !?|220445|
|                !? '|220446|
|                   #| 60626|
+--------------------+------+
only showing top 20 rows

+--------------------+------+
|                text|    id|
+--------------------+------+
|                   !|     0|
|                 ! '| 22935|
|                ! ''| 18235|
|              ! Alas|179257|
|         ! Brilliant| 22936|
|       ! Bril

In [53]:
from pyspark.sql.functions import lit
from pyspark.sql.functions import exp

# df = spark.createDataFrame([(1, "a", 23.0), (3, "B", -23.0)], ("x1", "x2", "x3"))
df_with_x4 = words2.withColumn("x4", lit(0))
#df_with_x4.show()


df_with_x5 = words2.withColumn("x5", words2.id %2 )
df_with_x5.show()

df_with_x5.write.mode("overwrite").parquet("test.parquet")

df_with_x5.write.partitionBy('x5').mode("overwrite").parquet("test.parquet")



+--------------------+------+---+
|                text|    id| x5|
+--------------------+------+---+
|                   !|     0|  0|
|                 ! '| 22935|  1|
|                ! ''| 18235|  1|
|              ! Alas|179257|  1|
|         ! Brilliant| 22936|  0|
|       ! Brilliant !| 40532|  0|
|     ! Brilliant ! '| 22937|  1|
|             ! C'mon| 60624|  0|
|! Gollum 's ` per...| 13402|  0|
|! Oh , look at th...|179258|  0|
|         ! Romething|140882|  0|
|               ! Run|179259|  1|
|         ! The Movie| 60625|  1|
|! The camera twir...|179260|  0|
|! True Hollywood ...|140883|  1|
|               ! Wow|179261|  1|
|            ! Zoom !|179262|  0|
|                  !?|220445|  1|
|                !? '|220446|  0|
|                   #| 60626|  0|
+--------------------+------+---+
only showing top 20 rows



In [ ]:
from pyspark.sql import Row
sc = spark.sparkContext

squaresDF = spark.createDataFrame(sc.parallelize(range(1, 6))
                                  .map(lambda i: Row(single=i, double=i ** 2)))
squaresDF.write.parquet("data/test_table/key=1")

# Create another DataFrame in a new partition directory,
# adding a new column and dropping an existing column
cubesDF = spark.createDataFrame(sc.parallelize(range(6, 11))
                                .map(lambda i: Row(single=i, triple=i ** 3)))
cubesDF.write.parquet("data/test_table/key=2")


In [ ]:

# Read the partitioned table
mergedDF = spark.read.option("mergeSchema", "true").parquet("data/test_table")
mergedDF.printSchema()
mergedDF.show()

#

In [ ]:
import numpy as np
import pandas as pd

# Enable Arrow-based columnar data transfers
spark.conf.set("spark.sql.execution.arrow.enabled", "true")

# Generate a Pandas DataFrame
pdf = pd.DataFrame(np.random.rand(100, 3))

# Create a Spark DataFrame from a Pandas DataFrame using Arrow
df = spark.createDataFrame(pdf)

# Convert the Spark DataFrame back to a Pandas DataFrame using Arrow
result_pdf = df.select("*").toPandas()
result_pdf.head()

In [ ]:
data = [1, 2, 3, 4, 5]
distData = sc.parallelize(data)
print(distData.reduce(lambda a, b: a + b))
print(distData.map(lambda a: a ** 2).reduce(lambda a,b: a+b))


In [ ]:
distFile = sc.textFile("../nlp/stanfordSentimentTreebank/dictionary*.txt")
distFile.map(lambda s: len(s)).reduce(lambda a, b: a + b)


In [ ]:
# from pyspark import SparkContext, SparkConf
# conf = SparkConf()
# conf.setMaster('spark://192.168.1.28:7077')
# conf.setAppName('mynewapp2')
# sc2 = SparkContext(conf=conf)
# print (sc2)

# def mod(x):
#     import numpy as np
#     return (x, np.mod(x, 2))
# rdd = sc2.parallelize(range(100)).map(mod).take(10)
# rdd